# Repressilator Example from Catalyst Documentation 

## Load Model

In [3]:
using GraphPlot
using Plots
using LinearAlgebra

In [4]:
using Catalyst
repressilator = @reaction_network Repressilator begin
       hillr(P₃,α,K,n), ∅ --> m₁
       hillr(P₁,α,K,n), ∅ --> m₂
       hillr(P₂,α,K,n), ∅ --> m₃
       (δ,γ), m₁ <--> ∅
       (δ,γ), m₂ <--> ∅
       (δ,γ), m₃ <--> ∅
       β, m₁ --> m₁ + P₁
       β, m₂ --> m₂ + P₂
       β, m₃ --> m₃ + P₃
       μ, P₁ --> ∅
       μ, P₂ --> ∅
       μ, P₃ --> ∅
end α K n δ γ β μ

Model Repressilator
States (6):
  m₁(t)
  m₂(t)
  m₃(t)
  P₁(t)
  P₂(t)
  P₃(t)
Parameters (7):
  α
  K
  n
  δ
  γ
  β
⋮

In [7]:
g=Graph(repressilator)

Graph("G", true, "dot", Catalyst.Statement[Catalyst.Node("m₁", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("m₂", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("m₃", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("P₁", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("P₂", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("P₃", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("rx_1", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "point", :color => "#E28F41", :width => ".1")), Catalyst.Node("r

## Matrix-Vector Reaction Rate Equation Representation

In [4]:
x=species(repressilator)

6-element Vector{Term{Real, Base.ImmutableDict{DataType, Any}}}:
 m₁(t)
 m₂(t)
 m₃(t)
 P₁(t)
 P₂(t)
 P₃(t)

In [5]:
N = netstoichmat(repressilator)

6×15 Matrix{Int64}:
 1  0  0  -1  1   0  0   0  0  0  0  0   0   0   0
 0  1  0   0  0  -1  1   0  0  0  0  0   0   0   0
 0  0  1   0  0   0  0  -1  1  0  0  0   0   0   0
 0  0  0   0  0   0  0   0  0  1  0  0  -1   0   0
 0  0  0   0  0   0  0   0  0  0  1  0   0  -1   0
 0  0  0   0  0   0  0   0  0  0  0  1   0   0  -1

In [6]:
rxs = reactions(repressilator)
ν = oderatelaw.(rxs)

15-element Vector{SymbolicUtils.Symbolic{Real}}:
 Catalyst.hillr(P₃(t), α, K, n)
 Catalyst.hillr(P₁(t), α, K, n)
 Catalyst.hillr(P₂(t), α, K, n)
 δ*m₁(t)
 γ
 δ*m₂(t)
 γ
 δ*m₃(t)
 γ
 β*m₁(t)
 β*m₂(t)
 β*m₃(t)
 μ*P₁(t)
 μ*P₂(t)
 μ*P₃(t)

In [7]:
osys = convert(ODESystem, repressilator)
odes = [eq.rhs for eq in equations(osys)]

6-element Vector{SymbolicUtils.Add{Real, Int64, Dict{Any, Number}, Nothing}}:
 γ + Catalyst.hillr(P₃(t), α, K, n) - δ*m₁(t)
 γ + Catalyst.hillr(P₁(t), α, K, n) - δ*m₂(t)
 γ + Catalyst.hillr(P₂(t), α, K, n) - δ*m₃(t)
 β*m₁(t) - μ*P₁(t)
 β*m₂(t) - μ*P₂(t)
 β*m₃(t) - μ*P₃(t)

In [8]:

odes2 = N * ν

6-element Vector{Any}:
 γ + Catalyst.hillr(P₃(t), α, K, n) - δ*m₁(t)
 γ + Catalyst.hillr(P₁(t), α, K, n) - δ*m₂(t)
 γ + Catalyst.hillr(P₂(t), α, K, n) - δ*m₃(t)
 β*m₁(t) - μ*P₁(t)
 β*m₂(t) - μ*P₂(t)
 β*m₃(t) - μ*P₃(t)

In [9]:
isequal(odes, odes2)

true

## Reaction Complex Representation

In [10]:
rn = @reaction_network begin
    k*A, 2*A + 3*B --> A + 2*C + D 
    b, C + D --> 2*A + 3*B 
end k b 

Model ##ReactionSystem#291
States (4):
  A(t)
  B(t)
  C(t)
  D(t)
Parameters (2):
  k
  b

In [11]:
species(rn)

4-element Vector{Term{Real, Base.ImmutableDict{DataType, Any}}}:
 A(t)
 B(t)
 C(t)
 D(t)

In [12]:
Z = complexstoichmat(rn)

4×3 Matrix{Int64}:
 2  1  0
 3  0  0
 0  2  1
 0  1  1

In [13]:
B = incidencemat(rn)

3×2 Matrix{Int64}:
 -1   1
  1   0
  0  -1

In [14]:
N = netstoichmat(rn)
N == Z*B

true

In [15]:
complexgraph(rn)

Graph("G", true, "dot", Catalyst.Statement[Catalyst.Node("<2A + 3B>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<2C + A + D>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<C + D>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Edge(Catalyst.NodeID[Catalyst.NodeID("<2A + 3B>", "", ""), Catalyst.NodeID("<2C + A + D>", "", "")], OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:color => "#d91111", :style => "dashed")), Catalyst.Edge(Catalyst.NodeID[Catalyst.NodeID("<C + D>", "", ""), Catalyst.NodeID("<2A + 3B>", "", "")], OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}())], OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:rankdir => "LR"), OrderedCollections.OrderedDict{Symbol, Union{Catal

In [27]:
complexgraph(repressilator)

Graph("G", true, "dot", Catalyst.Statement[Catalyst.Node("<∅>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<m₁>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<m₂>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<m₃>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<P₁ + m₁>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<P₂ + m₂>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<P₃ + m₃>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catal

## Aspects of Reaction Network Structure

In [28]:
rn = @reaction_network begin
    (k1, k2), A + B <--> C
    k3, C --> D+E
     (k4,k5), D+E <--> F
     (k6,k7), 2A <--> B+G
     k8, B+G --> H
     k9, H --> 2A
end k1 k2 k3 k4 k5 k6 k7 k8 k9

Model ##ReactionSystem#292
States (8):
  A(t)
  B(t)
  C(t)
  D(t)
⋮
Parameters (9):
  k1
  k2
  k3
  k4
⋮

In [29]:
complexgraph(rn)

Graph("G", true, "dot", Catalyst.Statement[Catalyst.Node("<A + B>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<C>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<D + E>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<F>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<2A>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<B + G>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("<H>", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Edge

## Linkage classes and sub-networks of the reaction network